## BRONZE TO SILVER LAYER

### Silver Layer - Stock Events History


In [1]:
# Importing Common Utility Funcation

import re

import pandas as pd
from ETLTools import GlobalPath

In [2]:
# Import necessary libraries and utility functions
%run ../COMMON/common_utility.ipynb

In [3]:
# Instantiate GlobalPath
stockdata_bronze_layer_path = GlobalPath("DATA/BRONZE/StockData")
stockevents_silver_file_path = GlobalPath(
    "DATA/SILVER/StockEvents/StockEvents_data.csv"
)

### Data Processing

This section handles the collection of data from the Bronze layer.

- Files are checked for availability.
- Data is read from each file, processed, and stored in a list of DataFrames.
- Process stock event data from the Bronze layer and transform it for the Silver layer.


In [4]:
# Initialize an empty list to store DataFrames
df_stock_events_list = []

# Generate file paths for available CSV files in the Bronze layer
file_paths = check_files_availability(
    stockdata_bronze_layer_path, file_pattern="*.csv"
)

# Process each CSV file found in the Bronze layer
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Extract the stock symbol by removing the date pattern from the file name
    df["symbol"] = re.sub(r"_\d{4}_\d{2}\.csv", "", file_path.name)

    # Append the DataFrame to the list
    df_stock_events_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_stock_events_list, ignore_index=True)

# Harmonize column names to ensure consistency
df = replace_punctuation_from_columns(df)

# Remove columns that contain only NA values
df.dropna(how="all", axis=1, inplace=True)

Number of Files Detected: 1018
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_02.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_03.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_04.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_05.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_07.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\AWL_2022_

Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_03.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_04.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_05.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_07.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DATAPATTNS_2022_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\DEEPAKNTR_2024_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\St

Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2022_12.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_01.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_02.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_03.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_04.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_05.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\HFCL_2023_07.csv
Processing file:

Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2022_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2022_09.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2022_10.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2022_11.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2022_12.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2023_01.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\PRAKASHSTL_2023_02.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\S

Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_05.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_07.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_09.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_10.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_11.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\SUZLON_2023_12.csv


Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_07.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_09.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_10.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_11.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2023_12.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2024_01.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\URJA_2024_02.csv
Processing file:

Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_02.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_03.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_04.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_05.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_06.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_07.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_08.csv
Processing file: D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\BRONZE\StockData\ZOMATO_2023_09.csv


In [5]:
# Convert datetime strings to date objects for consistency
df["date"] = pd.to_datetime(df["date"]).dt.date

# Pivot the DataFrame: columns become rows
df = df.melt(
    id_vars=["date", "symbol"],
    value_vars=["dividends", "stock_splits"],
    var_name="event",
    value_name="value",
)

# Convert event names to uppercase for uniformity
df["event"] = df["event"].str.upper()

# Round numerical values to 2 decimal places
df["value"] = df["value"].fillna(0)
df["value"] = df["value"].round(2)

# Filter out rows where the value is zero or less than a threshold
df = df[df["value"] > 0]

# Sort the DataFrame by stock name, date, and event for easier analysis
df = df.sort_values(by=["symbol", "date", "event"])

# Select relevant columns to export
df = df[["date", "symbol", "event", "value"]]

# Save the cleaned and transformed DataFrame as a CSV file in the Silver layer
df.to_csv(stockevents_silver_file_path, index=False)
print("SILVER Layer CSV file for Stock Events history successfully created at:")
print(stockevents_silver_file_path)

# Log the DataFrame debug for verification
df.info()

SILVER Layer CSV file for Stock Events history successfully created at:
D:\Study_Material\Anudip_foundation\Main_Projects\PortfolioTracker\DATA\SILVER\StockEvents\StockEvents_data.csv
<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 682 to 18837
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    44 non-null     object 
 1   symbol  44 non-null     object 
 2   event   44 non-null     object 
 3   value   44 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.7+ KB
